# VOTT \*.json to annotations

In [44]:
import numpy as np
import time
import sys
import os
import random
# from skimage import io
import pandas as pd
# from matplotlib import pyplot as plt
from shutil import copyfile

# import cv2
# import tensorflow as tf

import json
from pprint import pprint


In [60]:
base_path = '../faster-rcnn_vgg16/Dataset/airport'

In [46]:
df = pd.DataFrame(columns=['data_file_json', 'FileName', 'XMin', 'XMax', 'YMin', 'YMax', 'ClassName', 'Width','Height'])


for data_file_json in ['../data/labeling/data-merged.json','../data/labeling/data-filtered-yolo.json']:

    with open(data_file_json) as f:
        data = json.load(f)
    print("in {} found # keys: {}".format(data_file_json, len(data["frames"].keys())))
    
    i=1
    imax = len(data["frames"])

    for key,val in data["frames"].items():

        sys.stdout.write('Parse train_imgs ' + str(i) + '; Number of images: ' + str(len(data["frames"])) + '\r')
        sys.stdout.flush()

        for v in val:
            try:
                df = df.append({'data_file_json': data_file_json,
                                                    'FileName': key, 
                                                    'XMin': v["x1"], 
                                                    'XMax': v["x2"], 
                                                    'YMin': v["y1"], 
                                                    'YMax': v["y2"], 
                                                    'ClassName': v["tags"][0],
                                                    'Width': v["width"],
                                                    'Height': v["height"],}, 
                                                   ignore_index=True)
            except:
                continue
        i+=1

sys.stdout.write("\nFinal dataset has {} images".format(len(df)))

in ../data/labeling/data-merged.json found # keys: 339
in ../data/labeling/data-filtered-yolo.json found # keys: 449
Parse train_imgs 449; Number of images: 449
Final dataset has 1609 images

In [52]:
df.groupby("data_file_json").FileName.nunique()

data_file_json
../data/labeling/data-filtered-yolo.json    210
../data/labeling/data-merged.json           244
Name: FileName, dtype: int64

In [53]:
df_sorted = df.sort_values(by=['FileName'])
df_sorted.head(10)

,data_file_json,FileName,XMin,XMax,YMin,YMax,ClassName,Width,Height
1375,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00000_at_00-00-01.jpg,247.0,279.0,469.0,562.0,person,1280,720
924,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00001_at_00-00-01.jpg,205.0,259.0,522.0,662.0,person,1280,720
925,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00001_at_00-00-01.jpg,247.0,278.0,469.0,567.0,person,1280,720
1050,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00002_at_00-00-01.jpg,211.0,259.0,509.0,642.0,person,1280,720
1051,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00002_at_00-00-01.jpg,247.0,276.0,469.0,566.0,person,1280,720
1025,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00003_at_00-00-01.jpg,232.0,280.0,469.0,622.0,person,1280,720
1466,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00004_at_00-00-01.jpg,234.0,279.0,471.0,614.0,person,1280,720
1423,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00005_at_00-00-01.jpg,234.0,279.0,471.0,614.0,person,1280,720
1575,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00006_at_00-00-01.jpg,241.0,277.0,470.0,618.0,person,1280,720
1576,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00006_at_00-00-01.jpg,210.0,262.0,513.0,622.0,person,1280,720


### Split into train & test

In [54]:
df_sorted_grouped = df_sorted.groupby(['FileName']).count()

filenames_data = list(df_sorted_grouped.index)

random.shuffle(filenames_data)

msk = np.random.rand(len(filenames_data)) < 0.8
df_sorted_grouped["train_ind"] = msk

df_sorted_grouped = df_sorted_grouped[["train_ind"]]

In [55]:
dfx = df_sorted.merge(df_sorted_grouped,how="inner", on="FileName")
dfx.head()

,data_file_json,FileName,XMin,XMax,YMin,YMax,ClassName,Width,Height,train_ind
0,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00000_at_00-00-01.jpg,247.0,279.0,469.0,562.0,person,1280,720,True
1,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00001_at_00-00-01.jpg,205.0,259.0,522.0,662.0,person,1280,720,True
2,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00001_at_00-00-01.jpg,247.0,278.0,469.0,567.0,person,1280,720,True
3,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00002_at_00-00-01.jpg,211.0,259.0,509.0,642.0,person,1280,720,True
4,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00002_at_00-00-01.jpg,247.0,276.0,469.0,566.0,person,1280,720,True


In [56]:
# msk = np.random.rand(len(df)) < 0.8

# train = df[msk]
# test = df[~msk]

train = dfx[dfx.train_ind == True]
test = dfx[dfx.train_ind == False]

print("split to train: {} test: {} ~ train ratio: {:2f}".format(len(train), len(test), len(train)/ len(df)))

split to train: 1316 test: 293 ~ train ratio: 0.817899


In [57]:
train.head()

,data_file_json,FileName,XMin,XMax,YMin,YMax,ClassName,Width,Height,train_ind
0,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00000_at_00-00-01.jpg,247.0,279.0,469.0,562.0,person,1280,720,True
1,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00001_at_00-00-01.jpg,205.0,259.0,522.0,662.0,person,1280,720,True
2,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00001_at_00-00-01.jpg,247.0,278.0,469.0,567.0,person,1280,720,True
3,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00002_at_00-00-01.jpg,211.0,259.0,509.0,642.0,person,1280,720,True
4,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00002_at_00-00-01.jpg,247.0,276.0,469.0,566.0,person,1280,720,True


### Write  Train & Test annotation.txt and copy files

In [68]:
# print(train_df.head())
# train_df = pd.read_csv(os.path.join(base_path, 'train.csv'))

types = ["train","test"]

for t in types:
    
    if (t=="train"):
        tmp_df = train
    else:
        tmp_df = test
        
    if not os.path.exists(os.path.join(base_path, t)):
        os.makedirs(os.path.join(base_path, t))
        
    annotation_file = base_path + "/" + t + "_annotation.txt"
    f= open(annotation_file,"w+")
    
    for idx, row in tmp_df.iterrows():
        x1 = int(row['XMin'])
        x2 = int(row['XMax'])
        y1 = int(row['YMin'])
        y2 = int(row['YMax'])

        res_file_path = base_path + '/' + t
        fileName = os.path.join(res_file_path, row['FileName'])
        className = row['ClassName']
        f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + '\n')
#         try:
        copyfile(os.path.join(base_path,"all",row['FileName']), os.path.join(base_path,t,row['FileName']))
#         except:
#             print("file not exists...")
#             continue
    f.close()
    print("Output for {}: {}".format(t,annotation_file))

Output for train: ../faster-rcnn_vgg16/Dataset/airport/train_annotation.txt
Output for test: ../faster-rcnn_vgg16/Dataset/airport/test_annotation.txt


In [75]:
df_sorted_grouped.reset_index(level=0, inplace=True)
df_sorted_grouped.groupby("train_ind").FileName.nunique()

# df.groupby("data_file_json").FileName.nunique()

train_ind
False     51
True     215
Name: FileName, dtype: int64